## About my movie recommender demo  
[data source](https://www.kaggle.com/rounakbanik/the-movies-dataset/data)  
**Overview:** an ensemble data from
1. GroupLens(MovieLens): movie links and rating  
2. TMDB(The Movie DB) API: credits; keywords; metadata    

**Content:**  
45,000 movies in Full Movielens dataset, 26 million ratings from 270,000 users  
keywords/credits in the form of a string field JSON object


In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [25]:
DATA_DIR = '/Users/San/SelfLearning/RecommendationSystem/Movie_Recommendation/data/'
raw = pd.read_csv(os.path.join(DATA_DIR, 'movies_metadata.csv'), encoding='latin-1')

/Users/San/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
raw.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 24 columns):
adult                    45463 non-null bool
belongs_to_collection    4491 non-null object
budget                   45463 non-null int64
genres                   45463 non-null object
homepage                 7779 non-null object
id                       45463 non-null int64
imdb_id                  45446 non-null object
original_language        45452 non-null object
original_title           45463 non-null object
overview                 44509 non-null object
popularity               45463 non-null float64
poster_path              45080 non-null object
production_companies     45463 non-null object
production_countries     45463 non-null object
release_date             45379 non-null object
revenue                  45463 non-null int64
runtime                  45206 non-null float64
spoken_languages         45463 non-null object
status                   45382 non-null object
tag

In [29]:
raw.head(n=2).transpose()

,0,1
adult,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN
budget,30000000,65000000
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
homepage,http://toystory.disney.com/toy-story,NaN
id,862,8844
imdb_id,tt0114709,tt0113497
original_language,en,en
original_title,Toy Story,Jumanji
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...


In [30]:
raw.describe()

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4.546300e+04,45463.000000,45463.000000,4.546300e+04,45206.000000,45461.000000,45461.000000
mean,4.224579e+06,108359.918813,2.921378,1.120861e+07,94.127572,5.618178,109.895405
std,1.742413e+07,112460.749278,6.005234,6.433019e+07,38.406803,1.924205,491.305143
min,0.000000e+00,2.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,26449.500000,0.385945,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,60003.000000,1.127757,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,157328.000000,3.678588,0.000000e+00,107.000000,6.800000,34.000000
max,3.800000e+08,469172.000000,547.488298,2.787965e+09,1256.000000,10.000000,14075.000000


In [31]:
def describe_categorical(X):
    from IPython.display import display, HTML
    display(HTML(X[X.columns[X.dtypes == 'object']].describe().to_html()))
describe_categorical(raw) 

,belongs_to_collection,genres,homepage,imdb_id,original_language,original_title,overview,poster_path,production_companies,production_countries,release_date,spoken_languages,status,tagline,title,video
count,4491,45463,7779,45446,45452,45463,44509,45080,45463,45463,45379,45463,45382,20413,45461,45461
unique,1695,4066,7670,45416,89,43369,44306,45024,22710,2391,17170,1931,6,20284,42276,2
top,"{'id': 415931, 'name': 'The Bowery Boys', 'pos...","[{'id': 18, 'name': 'Drama'}]",http://www.georgecarlin.com,tt1180333,en,Hamlet,No overview found.,/5D7UBSEgdyONE6Lql6xS7s6OLcW.jpg,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",1/1/08,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Based on a true story.,Cinderella,False
freq,29,5000,12,3,32269,8,133,5,11875,17851,138,22397,45017,7,11,45368


In [32]:
def count_var_uniq_value(df):
    '''
    calculate unique value in each variable, and sort decendingly
    -----------
    :itype pd.DataFrame
    :rtype pd.Series
    '''
    import pandas as pd
    sort = pd.Series([len(df[x].value_counts().index) for x in df.columns], index=df.columns).sort_values(ascending=False)
    return sort
count_var_uniq_value(raw)

id                       45433
imdb_id                  45416
poster_path              45024
overview                 44306
popularity               43760
original_title           43369
title                    42276
production_companies     22710
tagline                  20284
release_date             17170
homepage                  7670
revenue                   6863
genres                    4066
production_countries      2391
spoken_languages          1931
vote_count                1820
belongs_to_collection     1695
budget                    1223
runtime                    353
vote_average                92
original_language           89
status                       6
video                        2
adult                        2
dtype: int64

In [34]:
def seperate_var(df, max_cat):
    '''
    seperate variables into boolean, multi-category and continous
    -----------------
    :itype
        df: pd.DataFrame
        max_cat: int (maximun number of unique values in multi-categorical variables)
    :rtype 3 lists
        three lists: v_boolean, v_multi, v_con
    '''
    v_boolean = []
    v_multi = []
    v_con = []
    for c in df.columns:
        num_value = len(df[c].value_counts().index)
        try:
            num_value >1
        except:
            print('column {} has less than 2 unique values'.format(c))
        if num_value <= 2:
            v_boolean.append(c)
        elif num_value <= max_cat:
            v_multi.append(c)
        else:
            v_con.append(c)
    print('boolean variable:\n', v_boolean, '\n')
    print("multi-categorical variables:\n", v_multi, '\n')     
    print("continuous/text variables:\n", v_con)

    return v_boolean, v_multi, v_con

bi, multi, con = seperate_var(raw, 89)

boolean variable:
 ['adult', 'video'] 

multi-categorical variables:
 ['original_language', 'status'] 

continuous/text variables:
 ['belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'tagline', 'title', 'vote_average', 'vote_count']


raw['year'] = pd.to_datetime(raw['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
raw.year.value_counts().plot()

Here we need to be aware that the majority of the movie data the demo use is recent movies, which also make sense to put more weight on recent movies than old movies to make recommendation

In [23]:
raw.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000
